## Introduction

Word2Vec is a popular algorithm used for generating dense vector representations of words in large corpora using unsupervised learning. The resulting vectors have been shown to capture semantic relationships between the corresponding words and are used extensively for many downstream natural language processing (NLP) tasks like sentiment analysis, named entity recognition and machine translation.  

SageMaker BlazingText which provides efficient implementations of Word2Vec on

- single CPU instance
- single instance with multiple GPUs - P2 or P3 instances
- multiple CPU instances (Distributed training)

In this notebook, we demonstrate how BlazingText can be used for distributed training of word2vec using multiple CPU instances.

## Setup

Let's start by specifying:
- The S3 buckets and prefixes that you want to use for saving model data and where training data is located. These should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region. 
- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the **get_execution_role** method from sagemaker python SDK.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker import s3
import boto3
import json

sess = sagemaker.Session()
role = get_execution_role()
print(role)

### Data Ingestion

BlazingText expects a single preprocessed text file with space separated tokens and each line of the file should contain a single sentence. In this example, let us train the vectors on [text8](http://mattmahoney.net/dc/textdata.html) dataset (100 MB), which is a small (already preprocessed) version of Wikipedia dump. Data is already downloaded from [matt mahoney's website](http://mattmahoney.net/dc/text8.zip), uncompressed and stored in `data_bucket`. 

In [ ]:
bucket = sess.default_bucket()

data_prefix = "blazingtext"
s3_train_data = f"s3://{bucket}/{data_prefix}/train"
s3_test_data = f"s3://{bucket}/{data_prefix}/test"

Next we need to setup an output location at S3, where the model artifact will be dumped. These artifacts are also the output of the algorithm's training job.

In [ ]:
output_prefix = "blazingtext"
s3_output_location = f"s3://{bucket}/{output_prefix}"

## Download Standardized dataset

As a prerequisite, we should have ran the [preprocessing notebook](../PreprocessNotebook.ipynb).

Then, we will create the variables with routes to the needed directories.

Finally, we will also download the dataset from S3.

In [ ]:
!mkdir -p data
data_directory = 'data'
test_dataset_file_name = '../data/preprocessed/sentiments-preprocessed-lemmatizing-test-10percent-dataset.csv'
train_dataset_file_name = '../data/preprocessed/sentiments-preprocessed-lemmatizing-train-10percent-dataset.csv'

In [ ]:
import csv

def convert_into_space_separated (dataset_file_name, destination_file_name):
    output_data = list()
    with open(dataset_file_name, newline='') as csv_file:
        input_data = list(csv.reader(csv_file))

        for message in input_data:
            message[0] = message[0].replace('1','_label_1')
            message[0] = message[0].replace('0','_label_0')
            output_data.append(f'{message[0]} {message[1]}\n')

    with open(f'{data_directory}/{destination_file_name}', mode='w') as output_file:
        for line in output_data:
            output_file.writelines(line)

convert_into_space_separated(train_dataset_file_name, 'train')
convert_into_space_separated(test_dataset_file_name, 'test')

In [ ]:
inputs = s3.S3Uploader.upload('data/train', s3_train_data)
inputs = s3.S3Uploader.upload('data/test', s3_test_data)

## Training Setup
Now that we are done with all the setup that is needed, we are ready to train our object detector. To begin, let us create a ``sageMaker.estimator.Estimator`` object. This estimator will launch the training job.

In [ ]:
region_name = boto3.Session().region_name

In [ ]:
container = sagemaker.image_uris.retrieve("blazingtext",region_name, "latest")
print(f"Using SageMaker BlazingText container: {container} ({region_name})")

## Training the BlazingText model for generating word vectors

Similar to the original implementation of [Word2Vec](https://arxiv.org/pdf/1301.3781.pdf), SageMaker BlazingText provides an efficient implementation of the continuous bag-of-words (CBOW) and skip-gram architectures using Negative Sampling, on CPUs and additionally on GPU[s]. The GPU implementation uses highly optimized CUDA kernels. To learn more, please refer to [*BlazingText: Scaling and Accelerating Word2Vec using Multiple GPUs*](https://dl.acm.org/citation.cfm?doid=3146347.3146354). BlazingText also supports learning of subword embeddings with CBOW and skip-gram modes. This enables BlazingText to generate vectors for out-of-vocabulary (OOV) words, as demonstrated in this [notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/blazingtext_word2vec_subwords_text8/blazingtext_word2vec_subwords_text8.ipynb).




Besides skip-gram and CBOW, SageMaker BlazingText also supports the "Batch Skipgram" mode, which uses efficient mini-batching and matrix-matrix operations ([BLAS Level 3 routines](https://software.intel.com/en-us/mkl-developer-reference-fortran-blas-level-3-routines)). This mode enables distributed word2vec training across multiple CPU nodes, allowing almost linear scale up of word2vec computation to process hundreds of millions of words per second. Please refer to [*Parallelizing Word2Vec in Shared and Distributed Memory*](https://arxiv.org/pdf/1604.04661.pdf) to learn more.

BlazingText also supports a *supervised* mode for text classification. It extends the FastText text classifier to leverage GPU acceleration using custom CUDA kernels. The model can be trained on more than a billion words in a couple of minutes using a multi-core CPU or a GPU, while achieving performance on par with the state-of-the-art deep learning text classification algorithms. For more information, please refer to [algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html) or [the text classification notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/blazingtext_text_classification_dbpedia/blazingtext_text_classification_dbpedia.ipynb).

To summarize, the following modes are supported by BlazingText on different types instances:

|          Modes         	| cbow (supports subwords training) 	| skipgram (supports subwords training) 	| batch_skipgram 	| supervised |
|:----------------------:	|:----:	|:--------:	|:--------------:	| :--------------:	|
|   Single CPU instance  	|   ✔  	|     ✔    	|        ✔       	|  ✔  |
|   Single GPU instance  	|   ✔  	|     ✔    	|                	|  ✔ (Instance with 1 GPU only)  |
| Multiple CPU instances 	|      	|          	|        ✔       	|     | |

Now, let's define the resource configuration and hyperparameters to train word vectors on *text8* dataset, using "batch_skipgram" mode on two c4.2xlarge instances.


In [ ]:
bt_model = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size=5,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_location,
    sagemaker_session=sess,
)

Please refer to [algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext_hyperparameters.html) for the complete list of hyperparameters.

In [ ]:
bt_model.set_hyperparameters(
    mode="supervised",
    epochs=5,
    min_count=5,
    sampling_threshold=0.0001,
    learning_rate=0.05,
    window_size=5,
    vector_dim=100,
    negative_samples=5,
    batch_size=11,  #  = (2*window_size + 1) (Preferred. Used only if mode is batch_skipgram)
    evaluation=True,  # Perform similarity evaluation on WS-353 dataset at the end of training
    subwords=False,
)  # Subword embedding learning is not supported by batch_skipgram

Now that the hyper-parameters are setup, let us prepare the handshake between our data channels and the algorithm. To do this, we need to create the `sagemaker.session.s3_input` objects from our data channels. These objects are then put in a simple dictionary, which the algorithm consumes.

In [ ]:
train_data = sagemaker.inputs.TrainingInput(
    s3_train_data, distribution="FullyReplicated", content_type="text/plain", s3_data_type="S3Prefix"
)
data_channels = {"train": train_data}

We have our `Estimator` object, we have set the hyper-parameters for this object and we have our data channels linked with the algorithm. The only  remaining thing to do is to train the algorithm. The following command will train the algorithm. Training the algorithm involves a few steps. Firstly, the instance that we requested while creating the `Estimator` classes is provisioned and is setup with the appropriate libraries. Then, the data from our channels are downloaded into the instance. Once this is done, the training job begins. The provisioning and data downloading will take some time, depending on the size of the data. Therefore it might be a few minutes before we start getting training logs for our training jobs. The data logs will also print out `Spearman's Rho` on some pre-selected validation datasets after the training job has executed. This metric is a proxy for the quality of the algorithm. 

Once the job has finished a "Job complete" message will be printed. The trained model can be found in the S3 bucket that was setup as `output_path` in the estimator.

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)

## Hosting / Inference
Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow us to make predictions (or inference) from the model. Note that we don't have to host on the same type of instance that we used to train. Because instance endpoints will be up and running for long, it's advisable to choose a cheaper instance for inference.

In [ ]:
bt_endpoint = bt_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

### Split the dataset into labels and messages

In [ ]:
import pandas as pd

df = pd.read_csv(f"{s3_train_data}/train")

labels = []
messages = []

for index, row in df.iterrows():
    tokens = row[0].split()
    label = tokens[0]
    words = tokens[1:]
    labels.append(label)
    messages.append(' '.join(words))

print(f'Labels length: {len(labels)}')
print(f'Messages length: {len(messages)}')
print(f'First label: {labels[0]}')
print(f'First message: {messages[0]}')

### Get a test subset of our dataset and obtain the metrics

Consider only a 30% of the messages.

Once we have our test dataset, use it to predict sentiment in batches our endpoint can handle.

Aggregate the predictions and compare with the labels.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

X_train, X_test, y_train, y_test = train_test_split(messages, labels, test_size=0.30)

x_batch = []
batch_size = 10
predicted_labels = []
for x in X_test:
    x_batch.append(x)
    if len(x_batch) == batch_size:
        payload = {"instances": x_batch}
        prediction = bt_endpoint.predict(json.dumps(payload),
                                         initial_args={"ContentType": "application/json", "Accept": "application/json"})

        prediction = pd.read_json(prediction)['label'].values.tolist()
        prediction = [x for item in prediction for x in item]
        predicted_labels.append(prediction)
        x_batch.clear()
predicted_labels = [x for item in predicted_labels for x in item]
print(f'Accuracy: {accuracy_score(y_test, predicted_labels)}')
print(f'F1: {f1_score(y_test, predicted_labels, pos_label="__label__1")}')
print(f'Precision: {precision_score(y_test, predicted_labels, pos_label="__label__1")}')
print(f'Recall Score: {recall_score(y_test, predicted_labels, pos_label="__label__1")}')


### Getting vector representations for words
#### Use JSON format for inference
The payload should contain a list of words with the key as "**instances**". BlazingText supports content-type `application/json`.

In [ ]:
words = ["today is gonna be the day"]

payload = {"instances": words}

response = bt_endpoint.predict(
    json.dumps(payload), initial_args={"ContentType": "application/json", "Accept": "application/json"}
)

vecs = json.loads(response)
print(vecs)

As expected, we get an n-dimensional vector (where n is vector_dim as specified in hyperparameters) for each of the words. If the word is not there in the training dataset, the model will return a vector of zeros.

### Stop / Close the Endpoint (Optional)
Finally, we should delete the endpoint before we close the notebook.

In [ ]:
sess.delete_endpoint(bt_endpoint.endpoint)